In [ ]:
%pip install transformers

In [43]:

from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn.functional as F
import random

def print_past_key_values(past_key_values):
    # links that roughly explains past_key_values:
    #  https://huggingface.co/docs/transformers/model_doc/t5
    # Better definitions:
    # The length of past_key_values is num_hidden_layers.
    # For each hidden layer, there are two elements. Each element is a tuple,
    #  representing either key or values.
    # Each key and value is a tensor of (batch_size, num_heads, sequence_length, head_size)。

    print("type of past_key_values: ", type(past_key_values))
    print("type of past_key_values[0]: ", type(past_key_values[0]))
    print("type of past_key_values[0][0]: ", type(past_key_values[0][0]))
    print("past_key_values shape: (",
          len(past_key_values), ", ",
          len(past_key_values[0]), ", ",
          past_key_values[0][0].shape, ")")
    print("More details on past_key_values shape: (",
          len(past_key_values), "-num_hidden_layers, ",
          len(past_key_values[0]), "-key/value, ",
          len(past_key_values[0][0]), "-batch_size, ",
          len(past_key_values[0][0][0]), "-num_heads, ",
          len(past_key_values[0][0][0][0]), "-sequence_length, ",
          len(past_key_values[0][0][0][0][0]), "-head_size)")


def generate_on_complete_context(model, tokenizer):
    print ("---------------generate_on_complete_context----------------------")
    # Encode the initial text
    ids = tokenizer.encode("Hello, my dog is big", return_tensors="pt")

    print("ids: ", ids)

    # Generate a continuation with the complete context
    output = model.generate(
        ids, max_new_tokens=20, num_return_sequences=1
    )
    print("output[0]: ", output[0])

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Generated text with complete context:", generated_text)

def genererate_using_past_key_values(model, tokenizer):
    print ("---------------genererate_using_past_key_values----------------------")

    # Encode the initial text
    ids = tokenizer.encode("Hello, my dog is big", return_tensors="pt")
    print("ids: ", ids)
    print("ids length: ", len(ids))

    # Split the input into two parts: "Hello, my dog is" and "big"
    context_ids = ids[:, :-1]
    next_word_ids = ids[:, -1:]
    print("context_ids: ", context_ids)
    print("next_word_ids: ", next_word_ids)

    # Generate a continuation using caching (past_key_values)
    output = model.generate(input_ids=context_ids, max_new_tokens = 1, use_cache=True, return_dict_in_generate=True)
    print("output[0]: ", output[0])
    past_key_values = output.past_key_values
    print_past_key_values(past_key_values)

    # Generate the next word based on the cached context
    # output = model(input_ids=next_word_ids, past_key_values=past_key_values)
    output = model.generate(input_ids=ids,  max_new_tokens = 10, past_key_values=past_key_values, return_dict_in_generate=True)
    print(output['sequences'][0])
    generated_text = tokenizer.decode(output['sequences'][0], skip_special_tokens=True)
    print("Generated text with complete context:", generated_text)

torch.manual_seed(42)
random.seed(42)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

generate_on_complete_context(model, tokenizer)
genererate_using_past_key_values(model, tokenizer)



---------------generate_on_complete_context----------------------
ids:  tensor([[15496,    11,   616,  3290,   318,  1263]])
output[0]:  tensor([15496,    11,   616,  3290,   318,  1263,   290,   314,  1101,   407,
         1654,   611,   339,   338,  1016,   284,   307,  1498,   284,  2513,
           13,   314,  1101,  1016,   284,   423])
Generated text with complete context: Hello, my dog is big and I'm not sure if he's going to be able to walk. I'm going to have
---------------genererate_using_past_key_values----------------------
ids:  tensor([[15496,    11,   616,  3290,   318,  1263]])
ids length:  1
context_ids:  tensor([[15496,    11,   616,  3290,   318]])
next_word_ids:  tensor([[1263]])
output[0]:  tensor([[15496,    11,   616,  3290,   318,   257]])
type of past_key_values:  <class 'tuple'>
type of past_key_values[0]:  <class 'tuple'>
type of past_key_values[0][0]:  <class 'torch.Tensor'>
past_key_values shape: ( 12 ,  2 ,  torch.Size([1, 12, 5, 64]) )
More details on pas